# MADRID CENTRAL

## Imports

Different libraries and functions we require to run the code

In [2]:
import pandas as pd
from matplotlib.path import Path
import numpy as np
import seaborn as sns
import time
from datetime import datetime as dt
import utm
import json
from tqdm import tqdm
import requests
import zipfile
import os
from IPython.display import display

# bokeh
import bokeh.palettes
from bokeh.plotting import figure, show, output_file, save, reset_output
from bokeh.io import output_notebook
from bokeh.models import HoverTool, Legend, ColumnDataSource, Span, ColorBar, DatetimeTickFormatter, CustomJS, Button
from bokeh.models.widgets import Panel, Tabs
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.layouts import layout
from bokeh.transform import linear_cmap


output_notebook()
np.random.seed(42)

Loading BokehJS ...

## Auxiliar Functions

In [3]:
def get_color_from_palette(color):
    """ Getting colors for plotting """
    return tuple([int(c * 255) for c in color])

def get_dark_color_from_palette(color):
    """ Getting darker colors for plotting """
    return tuple([int(c * 200) for c in color])

## Constants

Different constants to use through the code.

In [4]:
DOWNLOAD_DATA = False # Dowload or not the data
GENERATE_TRAFFIC_DATA = False # Generate traffic data

SAVE_PLOTS = True # If true, save the bokeh plots as html. Othewise, show

# Colors for the plots
PALETTE = "colorblind"

DISTRICT_COLORS = [
    get_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 21)
    ]

DISTRICT_DARK_COLORS = [
    get_dark_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 21)
    ]

MADRID_IN_OUT_COLORS = [
    get_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 2)
    ]

MADRID_IN_OUT_DARK_COLORS = [
    get_dark_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 2)
    ]

# Madrid Central key dates
START_MC = time.mktime(dt(2018, 11, 30, 0, 0, 0).timetuple())*1000
FINES_MC = time.mktime(dt(2019, 3, 15, 0, 0, 0).timetuple())*1000
END_MC = time.mktime(dt(2019, 7, 1, 0, 0, 0).timetuple())*1000


## 1. Motivation

The purpose of our project is to analyze the real impact of the measure that took place in the city center of Madrid called **Madrid Central**.

To sum up, Madrid Central was an environment measure that entered in force the **30th of November 2018**, it prohibited all polluting vehicles from entering the *"Centro"* district of Madrid. Unfortunately, it was taken down for political and economic reasons the **1st of July 2019**. 

In this study, we aim to evaluate the effectiveness of this vehicle regulation on pollution by analyzing traffic and air quality.

More on **Madrid Central** on the webpage.

<div align="center">
  <img src="https://s03.s3c.es/imag/_v0/864x1252/2/7/c/Madrid-Central_Plano-con-logo.jpg" alt="Madrid Central"/>
</div>

### The Dataset

We use multiple datasets, all of them from the **Open Data platform of the Town hall of Madrid**. 

<div align="center">
  <img src="https://www.zupimages.net/up/22/18/htou.png" alt="Open Data Madrid"/>
</div>


The datasets we use are:

 - **Air Quality Data**
    - [*Calidad del aire. Datos horarios desde 2001* (*Air quality. Hourly data since 2001*)](https://datos.madrid.es/sites/v/index.jsp?vgnextoid=f3c0f7d512273410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD):
    
      This data has in each file different air quality measures from multiple measurement stations. There are 24 stations all around the city of Madrid, with different gases measured in each one of them.

    - [*Calidad del aire. Estaciones de control* (*Air quality. Control stations*)](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=9e42c176313eb410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default):
    
      This is the geodata related to the measurement points. For plotting using Bokeh we had to change the format of how the spatial coordinates are presented.

 - **Traffic Data**
    - [*Tráfico. Histórico de datos del tráfico desde 2013* (*Traffic. Historic traffic data since 2013*)](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=33cb30c367e78410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default):
    
      This data has in separated files the recorded traffic measurements from more than 3000 points, taken every 15 minutes. This is the core dataset we use for our traffic analysis, but, to reduce the size of the data, we preprocess it to only use the average measure on each point per day.
    
    - [*Tráfico. Ubicación de los puntos de medida del tráfico* (*Traffic. Location of traffic measurement points*)](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=ee941ce6ba6d3410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD):
    
      This is the geodata related to the measurement points. For plotting using Bokeh we had to change the format of how the spatial coordinates are presented.

 - **Districts Geodata**

   - [*Distritos municipales de Madrid* (*Municipal districts of Madrid*)](https://datos.madrid.es/sites/v/index.jsp?vgnextoid=7d6e5eb0d73a7710VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD):
   
      The data has geodata of the limits of each of the districts of Madrid


### The Reason of the Dataset

We decided on these two datasets because we wanted to discover if **Madrid Central** achieved the objectives of reducing air pollution in the area, and we also wanted to inspect how effective and how big was the change in the traffic in and out of **Madrid Central**.

Additionally, we desired to analyze the relation between air quality and traffic in the city of Madrid.

### User Experience

While investigating this topic, we saw a lot of articles about **Madrid Central**. Some of them, saying how good and beneficial it was. Others, saying the exact opposite. Sadly, most of the news and articles were quite influenced by political ideas. This is because the measure was imposed by a political party, against the wishes of the opposing political party. Therefore, we feel they were biased, and did not rely on data and science.

We aim to fix this, trying to give an answer to how effective the measure really was, based on data, using visualizations and data analysis. We also want to present this in an informative and interactive way.

## 2. Basic Stats

### Data Processing

The first thing that has to be done in any data related task, is to preprocess and clean the data. In our case, this is very important, as we have as baseline many different files, with a lot of information. We have to reduce the dataset to obtain only the relevant parts.

**Air quality Data Processing**

**Traffic Data Processing**

The data we want to work with is very large, thus we need to download it from the source as it is not possible to upload it to the version control system we use (GitHub). 

In [5]:
def download_data_traffic():
    """ Download all traffic data from January 2016 (ID=32) until February 2020 (ID=81)
        Some files do not follow the same naming convention, and need repairing.
        The name convention that most files follow is '{num_month}-{num_year}.yaml',
        so everyone will follow that
    """
    FIRST_MONTH_ID = 32
    LAST_MONTH_ID = 81
    DATA_PATH = "data/traffic2"
    
    for month_id in tqdm(range(FIRST_MONTH_ID, LAST_MONTH_ID+1), desc="Downloading data", unit="file"):
        
        # Get month number, from 1 to 12
        current_month = ((month_id - FIRST_MONTH_ID) % 12) + 1
        
        # Get year number, from 2016 to 2021
        current_year = int((month_id - FIRST_MONTH_ID) / 12) + 2016    
               
        # If it has been downloaded already, skip it

        file_path = f"{DATA_PATH}/{current_month:02d}-{current_year}.csv"

        if not os.path.isfile(file_path):

            url = f"https://datos.madrid.es/egob/catalogo/208627-{month_id}-transporte-ptomedida-historico.zip"
            r = requests.get(url)
            z = zipfile.ZipFile(io.BytesIO(r.content))
            zipcsv = z.infolist()[-1]
            
            # Rename file
            zipcsv.filename = file_path
            
            # Extract file
            z.extract(zipcsv)
            

if DOWNLOAD_DATA:
    download_data_traffic()

Before diving into the actual data, we need to contextualize. Madrid is divided into districts. There are *21* one of them, being the area of **Madrid Central** exactly the same as the **Centro district** area (thus the name).

We have a dataset of where the measure of traffic points are located. As expected, they are not evenly distributed. Our first task is to see in which district each traffic measurement point is located.

In [6]:
traffic_points = pd.read_csv("shared_data/traffic_points/pmed_trafico_03052016.csv", sep=";")
traffic_points.head()

,idelem,tipo_elem,cod_cent,nombre,st_x,st_y
0,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06
1,3600,494,PM30901,PM30901,443729.047369,4.473268e+06
2,3705,494,PM41451,PM41451,439858.261097,4.471574e+06
3,6823,494,PM41453,PM41453,439188.095183,4.470895e+06
4,7033,495,01015,Pº Castellana S-N - Pl. Colon-Hermosilla,441569.555897,4.475502e+06


First we need to calculate the correct *utm* for displaying in `bokeh` maps.

In [7]:
def utm_from_latlon(lat, lon):
    """ From a given lat and lon, calculates the correct UTM coordinates to 
        plot using `bokeh` 
    """
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale

    return x, y

def get_lat_lon_utm(row):
    """ From a row containing the columns 'st_x' and 'st_y' calculates both the lat and lon
        and the correct UTM coordinates to plot using `bokeh`
    """

    # 30 and 'T' is the zone of Madrid
    lat, lon = utm.to_latlon(row["st_x"], row["st_y"], 30, "T")
    
    x, y = utm_from_latlon(lat, lon)

    return pd.Series([lat, lon, x, y])

In [8]:
traffic_points[["latitude", "longitude", "utm_x", "utm_y"]] = traffic_points.apply(get_lat_lon_utm, axis=1)
traffic_points = traffic_points.rename(columns = {'nombre':'name'})
traffic_points.head()

,idelem,tipo_elem,cod_cent,name,st_x,st_y,latitude,longitude,utm_x,utm_y
0,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06
1,3600,494,PM30901,PM30901,443729.047369,4.473268e+06,40.408129,-3.663184,-407783.811885,4.925429e+06
2,3705,494,PM41451,PM41451,439858.261097,4.471574e+06,40.392598,-3.708640,-412843.963659,4.923159e+06
3,6823,494,PM41453,PM41453,439188.095183,4.470895e+06,40.386431,-3.716471,-413715.710072,4.922257e+06
4,7033,495,01015,Pº Castellana S-N - Pl. Colon-Hermosilla,441569.555897,4.475502e+06,40.428107,-3.688839,-410639.639249,4.928350e+06


Then load the districts information to display them in the map.

In [9]:
with open("shared_data/districts/districts.geojson", "r") as geojson:
    geodata = json.load(geojson)

In [10]:
df_districts = pd.DataFrame([], columns=["name", "latitude",
                                         "longitude", "utm_x",
                                         "utm_y"])
for district in geodata["features"]:
    # Get district name
    district_name = district["properties"]["NOMBRE"]
    
    # Get district coordinates
    district_coord = district["geometry"]["coordinates"][0]
    df_district = pd.DataFrame(district["geometry"]["coordinates"][0], columns=["st_x", "st_y"])
    df_district["name"] = district_name
    
    # Calculate correct utm
    df_district[["latitude", "longitude", "utm_x", "utm_y"]] = df_district.apply(get_lat_lon_utm, axis=1)
    df_district = df_district.drop(columns=["st_x", "st_y"])
    
    # Append to all districts dataframe
    df_districts = pd.concat([df_districts, df_district]).reset_index(drop=True)


district_name = df_districts["name"].unique()
df_districts

,name,latitude,longitude,utm_x,utm_y
0,Centro,40.407345,-3.693162,-411120.867401,4925314.358763
1,Centro,40.407196,-3.693202,-411125.341089,4925292.526182
2,Centro,40.406986,-3.693227,-411128.197118,4925261.789915
3,Centro,40.407127,-3.693677,-411178.251532,4925282.481746
4,Centro,40.407256,-3.693849,-411197.420454,4925301.309521
...,...,...,...,...,...
9466,Moncloa - Aravaca,40.469899,-3.802628,-423306.659924,4934463.574953
9467,Moncloa - Aravaca,40.469823,-3.802359,-423276.645066,4934452.379956
9468,Moncloa - Aravaca,40.469748,-3.802093,-423247.03994,4934441.47116
9469,Moncloa - Aravaca,40.469672,-3.801822,-423216.881019,4934430.303684


Save in which district is each traffic point.

In [11]:
traffic_points["district"] = "None"
points = traffic_points[["utm_x", "utm_y"]]

for name in district_name:
    path = Path(df_districts[df_districts["name"] == name][["utm_x", "utm_y"]])
    points_in_path_mask = path.contains_points(points)
    traffic_points.loc[points_in_path_mask, "district"] = name

# Discard the traffic points outside any district of Madrid, as they are outside the city
    
traffic_points = traffic_points.drop(traffic_points[traffic_points["district"] == "None"].index)\
                .reset_index(drop=True)

traffic_points.head()

,idelem,tipo_elem,cod_cent,name,st_x,st_y,latitude,longitude,utm_x,utm_y,district
0,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06,Moncloa - Aravaca
1,3600,494,PM30901,PM30901,443729.047369,4.473268e+06,40.408129,-3.663184,-407783.811885,4.925429e+06,Moratalaz
2,3705,494,PM41451,PM41451,439858.261097,4.471574e+06,40.392598,-3.708640,-412843.963659,4.923159e+06,Carabanchel
3,6823,494,PM41453,PM41453,439188.095183,4.470895e+06,40.386431,-3.716471,-413715.710072,4.922257e+06,Carabanchel
4,7033,495,01015,Pº Castellana S-N - Pl. Colon-Hermosilla,441569.555897,4.475502e+06,40.428107,-3.688839,-410639.639249,4.928350e+06,Salamanca


The next step is to load the datasets for traffic information. This datasets have a lot of rows, as each of the more than 3000 measurement points record mutiple parameters each 15 minutes, so a rough approximation of how many rows each month file has is:

$$ 30(days) \cdot 24(hours) \cdot 4(measures\_per\_hour) \cdot 3000(traffic\_points) = 8640000 $$

And once again, if we take into account that we are using data from 2016 until the end of 2021, a more accurate row count would be:

$$ 4(years) \cdot 365(days) \cdot 24(hours) \cdot 4(measures\_per\_hour) \cdot 3000(traffic\_points) = 42048000 $$

This amount of data (more than 630 million rows) is too much to handle efficiently, and obtain relevant information. To reduce the amount of rows, we decide on keeping the average intensity of traffic (Number of cars) per day in each district. That way, we will have:

$$ 4(years) \cdot 365(days) \cdot 21(number\_districts) = 30660 $$

which is more manageable number, from where we aspire to detect the relevant information in the data. Around 13714 times less data.

In [12]:
def process_traffic_data(filepath, traffic_points_df):
    """ Function to process each traffic data file. This preoprocess has as objective to reduce
        the dimensionality od the data, only keeping one value per district per day, reducing this
        way the number of rows to handle.
        
        Arguments:
            filepath          -> path to load the csv
            traffic_points_df -> traffic_points dataset (where they are located)
    """
    
    # Load file
    traffic_df = pd.read_csv(filepath, sep=";")
    
    # For god knows why, there is one file that is separated by ',' instead of ';'
    # so we reread the file if it only has one column
    if len(traffic_df.columns) == 1:
        traffic_df = pd.read_csv(filepath, sep=",")
    
    # If the 'idelem' column does not exists, is because is called 'id', so rename column
    if "idelem" not in traffic_df.columns:
        traffic_df = traffic_df.rename(columns = {'id':'idelem'})
    
    # Use only the traffic points for whom we have information 
    traffic_df = traffic_df[traffic_df["idelem"].isin(traffic_points_df["idelem"])]
    
    # Transform date to datime type
    traffic_df["fecha"] = pd.to_datetime(traffic_df["fecha"])
    
    # Get date in separate columns
    traffic_df["day"] = traffic_df["fecha"].dt.day
    traffic_df["month"] = traffic_df["fecha"].dt.month
    traffic_df["year"] = traffic_df["fecha"].dt.year

    # Group by id and date, up to day, and get the average intensity perr traffic point
    traffic_df = traffic_df.groupby(["idelem",
                                     "day",
                                     "month",
                                     "year"]).agg(mean_intensity=("intensidad", "mean")).reset_index()
    
    # Merge with the traffic points to get the district for each point
    traffic_df = traffic_df.merge(traffic_points_df[["idelem", "district"]], on="idelem")
    
    # Save also the traffic points as separated files
    traffic_points_intensity_df = traffic_df.copy()

    # Group by again, to get only one value per district per day
    traffic_df = traffic_df.groupby(["district", "day", "month", "year"]).mean()["mean_intensity"].reset_index()
    
    # Get the date and day of the week for plotting purpose
    traffic_df["date"] = pd.to_datetime(traffic_df[["day", "month", "year"]])
    traffic_df["day_of_week"] = traffic_df["date"].dt.day_name()

    traffic_points_intensity_df["date"] = pd.to_datetime(traffic_points_intensity_df[["day", "month", "year"]])
    traffic_points_intensity_df["day_of_week"] = traffic_points_intensity_df["date"].dt.day_name()

    # Save idelem as integer
    traffic_points_intensity_df["idelem"] = traffic_points_intensity_df["idelem"].astype(int)
    
    return traffic_df, traffic_points_intensity_df

In [13]:
def load_all_trafic_data(traffic_points_df):
    """ Function to load all trafic data from the data folder,
        after being processed
        
        Arguments:
            traffic_points_df -> traffic_points dataset (where they are located)
    """
    
    DATA_PATH = "data/traffic"
    
    traffic_data = pd.DataFrame([], columns=["district", "date", "day_of_week",
                                             "day", "month", "year", "mean_intensity"])
    
    traffic_points_data = pd.DataFrame([], columns=["district", "date", "day_of_week",
                                             "day", "month", "year", "mean_intensity"])
    
    for filepath in tqdm(os.listdir(DATA_PATH), desc="Processing files", unit="file"):
        traffic_df, traffic_points_intensity_df= process_traffic_data(os.path.join(DATA_PATH, filepath), traffic_points_df)
        
        traffic_data = pd.concat([traffic_data, traffic_df])
        traffic_points_data = pd.concat([traffic_points_data, traffic_points_intensity_df])


    return (traffic_data.sort_values(by=["district", "date"]).reset_index(drop="True"),
            traffic_points_data.sort_values(by=["district", "date", "idelem"]).reset_index(drop="True"))

In [14]:
df_traffic_path = "shared_data/traffic_intensity.csv"
df_traffic_intensity_path = "data/traffic_points_intensity.csv"


if not GENERATE_TRAFFIC_DATA:
    total_traffic_df = pd.read_csv(df_traffic_path)
    total_traffic_df["date"] = pd.to_datetime(total_traffic_df["date"])

    traffic_points_intensity_df = pd.read_csv(df_traffic_intensity_path)
    traffic_points_intensity_df["date"] = pd.to_datetime(traffic_points_intensity_df["date"])
    traffic_points_intensity_df["idelem"] = traffic_points_intensity_df["idelem"].astype(int)
else:
    total_traffic_df, traffic_points_intensity_df = load_all_trafic_data(traffic_points)

    total_traffic_df.to_csv(df_traffic_path, index=False)
    traffic_points_intensity_df.to_csv(df_traffic_intensity_path, index=False)

In [15]:
total_traffic_df.head()

,district,date,day_of_week,day,month,year,mean_intensity
0,Arganzuela,2016-01-01,Friday,1,1,2016,397.162405
1,Arganzuela,2016-01-02,Saturday,2,1,2016,433.307174
2,Arganzuela,2016-01-03,Sunday,3,1,2016,401.376015
3,Arganzuela,2016-01-04,Monday,4,1,2016,531.390601
4,Arganzuela,2016-01-05,Tuesday,5,1,2016,524.791899


In [16]:
traffic_points_intensity_df.head()

,district,date,day_of_week,day,month,year,mean_intensity,idelem
0,Arganzuela,2016-01-01,Friday,1,1,2016,1650.677083,1019
1,Arganzuela,2016-01-01,Friday,1,1,2016,1659.406250,1020
2,Arganzuela,2016-01-01,Friday,1,1,2016,1015.552083,1021
3,Arganzuela,2016-01-01,Friday,1,1,2016,403.718750,1022
4,Arganzuela,2016-01-01,Friday,1,1,2016,462.864583,1023


### Exploratory data analysis

**Air quality data exploration**

**Traffic data exploration**

We want to check first how many traffic points are in each district.

In [17]:
traffic_points.groupby("district").agg(number_traffic_points=("idelem", "count")).sort_values("number_traffic_points").reset_index()

,district,number_traffic_points
0,Barajas,41
1,Vicálvaro,45
2,Villa de Vallecas,63
3,Villaverde,77
4,Moratalaz,114
5,Usera,121
6,Chamberí,134
7,Retiro,134
8,Puente de Vallecas,137
9,Centro,176


From the table we can see that there is quite a big difference between the different traffic points, being the district with the least number *Barajas* (41) and the district with the biggest number *Chamartín* (357).

This is something we have to take into account, as too few datapoints may result in poor conclusions. Luckly, the **Centro** district has **176** measurement points, which we believe it's enough given the small area (5.23 km²) of this district.

Now let's compare the traffic intensity in each district, to see which districts are busier.

In [18]:
df_traffic_points = traffic_points.copy()
df_traffic_points = pd.merge(df_traffic_points,
                             traffic_points_intensity_df.groupby("idelem").agg(intensity=("mean_intensity", "mean")).reset_index(),
                             on="idelem")

df_traffic_points.head()

,idelem,tipo_elem,cod_cent,name,st_x,st_y,latitude,longitude,utm_x,utm_y,district,intensity
0,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06,Moncloa - Aravaca,635.772220
1,3600,494,PM30901,PM30901,443729.047369,4.473268e+06,40.408129,-3.663184,-407783.811885,4.925429e+06,Moratalaz,892.839662
2,3705,494,PM41451,PM41451,439858.261097,4.471574e+06,40.392598,-3.708640,-412843.963659,4.923159e+06,Carabanchel,1877.529488
3,6823,494,PM41453,PM41453,439188.095183,4.470895e+06,40.386431,-3.716471,-413715.710072,4.922257e+06,Carabanchel,1720.147974
4,7033,495,01015,Pº Castellana S-N - Pl. Colon-Hermosilla,441569.555897,4.475502e+06,40.428107,-3.688839,-410639.639249,4.928350e+06,Salamanca,1033.697074


In [19]:
p = figure(title="Average Intensity per District", x_axis_type="mercator", y_axis_type="mercator",
           height=600, width=600, tools="")

p.axis.visible = False
p.toolbar.logo = None
p.toolbar_location = None


for name in district_name:
    # Districts
    source_dict = dict(utm_x = [[x for x in df_districts[df_districts["name"] == name]["utm_x"]]],
                       utm_y = [[y for y in df_districts[df_districts["name"] == name]["utm_y"]]],
                       name = [name],
                       intensity = [total_traffic_df[total_traffic_df["district"] == name]["mean_intensity"].mean()])

    source = ColumnDataSource(source_dict)
    p.patches(xs="utm_x", ys="utm_y", color=linear_cmap("intensity", "Viridis256", 100, 700), line_width=3, alpha=0.4, 
            source=source, muted=False, muted_alpha=0.1)

    

source = ColumnDataSource(df_traffic_points)

p.circle(x="utm_x", y="utm_y", line_width=1, color=linear_cmap("intensity", "Viridis256", 100, 700),
    source=source, alpha=0.75, size=3)
    

cartodb = get_provider(CARTODBPOSITRON)
p.add_tile(cartodb)

TOOLTIPS = [
    ("District Name", "@name"),
    ("Average Intensity", "@intensity"),
]
p.add_tools(HoverTool(tooltips=TOOLTIPS))

p.background_fill_color = None
p.border_fill_color = None

mapper = linear_cmap(field_name='intensity', palette="Viridis256", low=100, high=700)

color_bar = ColorBar(color_mapper=mapper['transform'], width=8)

p.add_layout(color_bar, 'right')

p = layout(p, sizing_mode='scale_both')

if SAVE_PLOTS:
    output_file("html_plots/average_intensity_districts.html", title="Average Intensity Districts Map")
    save(p)

    reset_output()

    output_notebook()
else:
    show(p)

Loading BokehJS ...

From this plot, we can see that **Centro** is not the most transited district. We do not know yet if it is thanks to the measures, or that it is just not a very transited district. What we do know for sure, and that can be also powered by the measures, the surrounded districts, such as **Arganzuela**, **Retiro** or **Moncloa** are some of the busiest districts in the city, so it will be interesting to see if there is a border effect because of **Madrid Central**.

If we focus more on the traffic points as separated measures instead of districts as a whole, we can observe an expected result. The traffic intensity in the main roads of Madrid is higher than in the secondary roads. We can see that, inside **Madrid Central**, the most crowded road is ***Gran Via***, which is the main commercial road in the district. We also appreciate a lot of traffic surrounding **Madrid Central**, which again may be interesting to investigate if it is a normal traffic flow, or it is because of the measures. 

Doing the average it is not enough for doing a time series analysis, thus we are going to see how does the traffic intensity per district looks like per day, from 2016 to February 2020. In order to be able to see the evolution of the traffic during time.

In [20]:
# Plot traffic intensity through time by district
p = figure(title="Traffic intensity through time by district", x_axis_label="Date",
           y_axis_label="Traffic intensity", height=200, width=400)

fig_lines = []

for name, color in zip(district_name, DISTRICT_COLORS):
    source = ColumnDataSource(total_traffic_df[total_traffic_df["district"] == name])
    l = p.line(x="date", y="mean_intensity", source=source,
               color=color, legend_label=name, visible=True,
               line_width=3, alpha=0.8)
    fig_lines.append(l)
    
p.renderers.extend(fig_lines)

    
p.add_layout(p.legend[0], "right")
p.legend.click_policy = "hide"

p.xaxis.formatter=DatetimeTickFormatter(
        days=['%a %d/%m/%Y'],
        months=['%b %Y'],
        years = ['%Y']
    )

# Hover tooltip
TOOLTIPS = [
    ("District", "@district"),
    ("Intensity", "@mean_intensity"),
    ("Day", "@day_of_week @day/@month/@year")
]
p.add_tools(HoverTool(tooltips=TOOLTIPS, mode="vline"))

# Button
button = Button(
    label="Switch all lines visibility", button_type="success", max_width=100, max_height=50
)
callback = CustomJS(args=dict(lines=fig_lines),
    code="""
    for(var i=0; i<lines.length; i++){
        lines[i].visible = !lines[i].visible;
    }
    """
)
button.js_on_click(callback)


p = layout(p, button, sizing_mode="scale_both")


if SAVE_PLOTS:
    output_file("html_plots/traffict_intensity_time_district.html", title="Traffic intensity through time by district")
    save(p)

    reset_output()

    output_notebook()
else:
    show(p)

Loading BokehJS ...

Given the fact that we have 21 districts, the plot becomes a bit overwhelm. But if we show only the district/s that we want to see or compare, we can perceive the progress of the traffic intensity along the days. We detect that **Centro** is not the district with more traffic, or that almost all the districts follow the same tendencies. When there is a day that has less traffic, we see the decrement in all the districts in proportion to its own intensity.

We suspect that these days when the traffic increases or decreases in all districts are festive or something is celebrated. Then let's try to find out if those days are related to any specific dates.

In [21]:
# Festivities of Madrid
festive_days = [
    [1, 1, "New Year"],
    [1, 6, "Epiphany of the Lord"],
    [5, 1, "Labor Day"],
    [5, 2, "Day of the Community of Madrid"],
    [5, 15, "San Isidro"],
    [7, 25, "Santiago Apóstol"],
    [8, 6, "Fiestas de San Cayetano"],
    [8, 15, "Asunción de la Virgen"],
    [10, 12, "National Day of Spain"],
    [11, 1, "All Saints' Day"],
    [11, 9, "Días de la Almudena."],
    [12, 6, "Day of the Constitution"],
    [12, 8, "Inmaculada Conceptción"],
    [12, 25, "Christmas"]
]

months_name = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

festivities = pd.DataFrame(festive_days, columns=["month", "day", "festivity"])

festivities["month_name"] = festivities.apply(lambda x: months_name[int(x["month"]-1)], axis=1)


festivities["date"] = festivities["day"].astype(str) + " of " + festivities["month_name"].astype(str)

festivities

,month,day,festivity,month_name,date
0,1,1,New Year,January,1 of January
1,1,6,Epiphany of the Lord,January,6 of January
2,5,1,Labor Day,May,1 of May
3,5,2,Day of the Community of Madrid,May,2 of May
4,5,15,San Isidro,May,15 of May
5,7,25,Santiago Apóstol,July,25 of July
6,8,6,Fiestas de San Cayetano,August,6 of August
7,8,15,Asunción de la Virgen,August,15 of August
8,10,12,National Day of Spain,October,12 of October
9,11,1,All Saints' Day,November,1 of November


In [22]:
# Only Centro district
df_centro = total_traffic_df[total_traffic_df["district"] == "Centro"].reset_index(drop=True)
df_centro

,district,date,day_of_week,day,month,year,mean_intensity
0,Centro,2016-01-01,Friday,1,1,2016,309.436629
1,Centro,2016-01-02,Saturday,2,1,2016,361.187308
2,Centro,2016-01-03,Sunday,3,1,2016,373.553736
3,Centro,2016-01-04,Monday,4,1,2016,421.640004
4,Centro,2016-01-05,Tuesday,5,1,2016,388.572838
...,...,...,...,...,...,...,...
1514,Centro,2020-02-25,Tuesday,25,2,2020,405.391449
1515,Centro,2020-02-26,Wednesday,26,2,2020,417.048690
1516,Centro,2020-02-27,Thursday,27,2,2020,431.823006
1517,Centro,2020-02-28,Friday,28,2,2020,432.899723


In [23]:
# Centro 
df_centro_year = df_centro.groupby(["year", "month", "day"]).agg(mean_intensity_year = ("mean_intensity", "mean")).reset_index()
df_centro_year["date"] = pd.to_datetime(df_centro_year[["year", "month", "day"]])

df_centro_year = pd.merge(df_centro_year, festivities[["month", "day", "festivity"]], on=["month", "day"], how="left")
df_centro_year.loc[df_centro_year["festivity"].isna(), "festivity"] = "None"

# All districts
df_year = total_traffic_df.groupby(["year", "month", "day"]).agg(mean_intensity_year = ("mean_intensity", "mean")).reset_index()
df_year["date"] = pd.to_datetime(df_year[["year", "month", "day"]])

df_year = pd.merge(df_year, festivities[["month", "day", "festivity"]], on=["month", "day"], how="left")
df_year.loc[df_year["festivity"].isna(), "festivity"] = "None"

display(df_centro_year)
df_year

,year,month,day,mean_intensity_year,date,festivity
0,2016,1,1,309.436629,2016-01-01,New Year
1,2016,1,2,361.187308,2016-01-02,None
2,2016,1,3,373.553736,2016-01-03,None
3,2016,1,4,421.640004,2016-01-04,None
4,2016,1,5,388.572838,2016-01-05,None
...,...,...,...,...,...,...
1514,2020,2,25,405.391449,2020-02-25,None
1515,2020,2,26,417.048690,2020-02-26,None
1516,2020,2,27,431.823006,2020-02-27,None
1517,2020,2,28,432.899723,2020-02-28,None


,year,month,day,mean_intensity_year,date,festivity
0,2016,1,1,255.887737,2016-01-01,New Year
1,2016,1,2,288.450196,2016-01-02,None
2,2016,1,3,261.173685,2016-01-03,None
3,2016,1,4,347.221850,2016-01-04,None
4,2016,1,5,342.530852,2016-01-05,None
...,...,...,...,...,...,...
1514,2020,2,25,448.332289,2020-02-25,None
1515,2020,2,26,458.179619,2020-02-26,None
1516,2020,2,27,467.711812,2020-02-27,None
1517,2020,2,28,453.818640,2020-02-28,None


In [24]:
# Create annotations for time marks
startMC_span = Span(location=START_MC,
                              dimension='height', line_color='black',
                              line_dash='dashed', line_width=2, line_alpha=0.3)

finesMC_span = Span(location=FINES_MC,
                              dimension='height', line_color='black',
                              line_dash='dashed', line_width=2, line_alpha=0.3)

endMC_span = Span(location=END_MC,
                              dimension='height', line_color='black',
                              line_dash='dashed', line_width=2, line_alpha=0.3)


# Plot
p = figure(title="Average traffic intensity per day through the years in Madrid Central and in the city of Madrid", x_axis_label="Year",
           y_axis_label="Traffic intensity average by day", width=1100)

source = ColumnDataSource(df_centro_year)
p.vbar(x="date", top="mean_intensity_year", source=source,  width=0.99, alpha=0.2,
       line_alpha=1, line_color=DISTRICT_DARK_COLORS[0],
       legend_label="Madrid Central", color=DISTRICT_COLORS[0])


source = ColumnDataSource(df_centro_year[df_centro_year["festivity"] != "None"])
p.vbar(x="date", top="mean_intensity_year", source=source, width=1,
       line_color=DISTRICT_DARK_COLORS[3],
       legend_label="Festive days in Madrid", color=DISTRICT_COLORS[3])
       

source = ColumnDataSource(df_year)
p.line(x="date", y="mean_intensity_year", source=source, line_width=3, alpha=0.6,
       legend_label="Madrid", color=DISTRICT_COLORS[1])


p.xaxis.formatter=DatetimeTickFormatter(
        days=['%a %d/%m/%Y'],
        months=['%b %Y'],
        years = ['%Y']
    )

# add annotations to plot
p.add_layout(startMC_span)
p.add_layout(finesMC_span)
p.add_layout(endMC_span)

# Hover tooltip
TOOLTIPS = [
    ("Average Intensity per day", "@mean_intensity_year"),
    ("Date", "@day/@month/@year"),
    ("Festivity", "@festivity")
    ]
p.add_tools(HoverTool(tooltips=TOOLTIPS, mode="vline"))

p.add_layout(p.legend[0], "right")

if SAVE_PLOTS:
    output_file("html_plots/traffict_intensity_time_in_out.html", title="Traffic intensity through time in and out Madrid Central")
    save(p)

    reset_output()

    output_notebook()
else:
    show(p)

Loading BokehJS ...

Right as we thought, almost all the peaks and valleys coincide with festive days in Madrid. Which gives us an overview of the traffic behavior. We also observe that the area of Madrid Central (**Centro** district) follows the same average as the whole city of Madrid. 

The gray dashed lines represent the 3 key dates (start of Madrid Central, start of fines, end of Madrid Central).

Once we have seen the whole picture of the traffic intensity evolution during the period we are investigating, we want to focus on a **yearly**, **monthly** and **weekly** analysis between **Madrid Central** area and the city of **Madrid**.

In [25]:
# Yearly analysis
df_centro_year = df_centro.groupby("year").agg(mean_intensity_year = ("mean_intensity", "mean")).reset_index()

df_year = total_traffic_df.groupby("year").agg(mean_intensity_year = ("mean_intensity", "mean")).reset_index()

display(df_centro_year)
df_year

,year,mean_intensity_year
0,2016,424.935031
1,2017,423.296288
2,2018,395.115073
3,2019,387.492076
4,2020,385.513792


,year,mean_intensity_year
0,2016,402.843768
1,2017,395.590693
2,2018,400.240504
3,2019,397.607394
4,2020,409.064831


In [40]:
# Plot yearly analysis
p = figure(title="Average traffic intensity per day through the years in Madrid Central and in the city of Madrid", x_axis_label="Year",
           y_axis_label="Traffic intensity average by day", height=400, width=800)

source = ColumnDataSource(df_centro_year)
p.vbar(x="year", top="mean_intensity_year", source=source, width=0.5, legend_label="Madrid Central", color=DISTRICT_COLORS[0])

source = ColumnDataSource(df_year)
p.line(x="year", y="mean_intensity_year", source=source, line_width=3, legend_label="Madrid", color=DISTRICT_COLORS[1])

# Hover tooltip
TOOLTIPS = [
    ("Average Intensity per day", "@mean_intensity_year"),
    ("Year", "@year")
    ]
p.add_tools(HoverTool(tooltips=TOOLTIPS, mode="vline"))

p.add_layout(p.legend[0], "right")

if SAVE_PLOTS:
    output_file("html_plots/yearly_analysis_in_out.html", title="Average traffic intensity per day through the years in and out of Madrid Central")
    save(p)

    reset_output()

    output_notebook()
else:
    show(p)

Loading BokehJS ...

From this first plot we see that in 2016 and 2017 the average of traffic intensity per day was higher in Madrid Central than in the rest of the city, but in 2018 it changed. **Since 2018**, we observe how the **average** from **Madrid Central** decreases and becomes **lower** than the average from the city. 

In [45]:
# Monthly analysis
months_name = ["Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sept", "Oct", "Nov", "Dec"]

df_centro_month = df_centro.groupby("month").agg(mean_intensity_month = ("mean_intensity","mean")).reset_index()
df_month = total_traffic_df.groupby("month").agg(mean_intensity_month = ("mean_intensity", "mean")).reset_index()

df_centro_month["month_name"] = df_centro_month.apply(lambda x: months_name[int(x["month"]-1)], axis=1)
df_month["month_name"] = df_month.apply(lambda x: months_name[int(x["month"]-1)], axis=1)

display(df_centro_month)                                                                         
df_month

,month,mean_intensity_month,month_name
0,1,390.225333,Jan
1,2,422.934982,Feb
2,3,423.078660,Mar
3,4,417.780030,Apr
4,5,417.368505,May
5,6,429.546765,June
6,7,390.347374,July
7,8,324.546668,Aug
8,9,419.914969,Sept
9,10,425.815269,Oct


,month,mean_intensity_month,month_name
0,1,393.485793,Jan
1,2,417.557986,Feb
2,3,409.181076,Mar
3,4,408.456585,Apr
4,5,415.059586,May
5,6,428.255877,June
6,7,375.828154,July
7,8,291.408997,Aug
8,9,409.447912,Sept
9,10,422.270867,Oct


In [46]:
# Plot monthly analysis
p = figure(title="Average traffic intensity per day in a month in Madrid Central and in the city of Madrid", x_axis_label="Month",
           y_axis_label="Traffic intensity average by day", height=400, width=800, x_range=months_name)

source = ColumnDataSource(df_centro_month)
p.vbar(x="month_name", top="mean_intensity_month", source=source, width=0.5, legend_label="Madrid Central", color=DISTRICT_COLORS[0])

source = ColumnDataSource(df_month)
p.line(x="month_name", y="mean_intensity_month", source=source, line_width=3, legend_label="Madrid", color=DISTRICT_COLORS[1])

# Hover tooltip
TOOLTIPS = [
    ("Average Intensity per day", "@mean_intensity_month"),
    ("Month", "@month_name")
    ]
p.add_tools(HoverTool(tooltips=TOOLTIPS, mode="vline"))

p.add_layout(p.legend[0], "right")

if SAVE_PLOTS:
    output_file("html_plots/monthly_analysis_in_out.html", title="Average traffic intensity per day in a month in and out of Madrid Central")
    save(p)

    reset_output()

    output_notebook()
else:
    show(p)

Loading BokehJS ...

Regarding the average of traffic intensity per day in a month, we see that it is more or less the same for Madrid Central and the city. Still worth noting that for some months the average from Madrid Central is a bit higher. For example during summer holidays (June, July, August, September) or Easter holidays (Match-April depending on the year) which could be related to more tourists going to the city center (Madrid Central area).

In [36]:
# Weekly analysis
week_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

df_centro_week = df_centro.groupby("day_of_week").agg(mean_intensity_week = ("mean_intensity", "mean")).reset_index()
df_week = total_traffic_df.groupby("day_of_week").agg(mean_intensity_week = ("mean_intensity", "mean")).reset_index()

df_centro_week["day_of_week"] = df_centro_week["day_of_week"].astype("category")
df_centro_week["day_of_week"] = df_centro_week["day_of_week"].cat.set_categories(week_days)

df_centro_week = df_centro_week.sort_values("day_of_week")

df_week["day_of_week"] = df_week["day_of_week"].astype("category")
df_week["day_of_week"] = df_week["day_of_week"].cat.set_categories(week_days)

df_week = df_week.sort_values("day_of_week")

display(df_centro_week)
df_week

,day_of_week,mean_intensity_week
1,Monday,404.781332
5,Tuesday,414.155558
6,Wednesday,423.733646
4,Thursday,434.114738
0,Friday,445.293051
2,Saturday,373.782265
3,Sunday,351.353999


,day_of_week,mean_intensity_week
1,Monday,419.675982
5,Tuesday,429.313752
6,Wednesday,435.721548
4,Thursday,442.083065
0,Friday,444.174894
2,Saturday,327.552783
3,Sunday,297.044891


In [47]:
# Plot weekly analysis
p = figure(title="Average traffic intensity per day in a week in Madrid Central and in the city of Madrid", x_axis_label="Weekday",
           y_axis_label="Traffic intensity average by day", height=400, width=800, x_range=week_days)

source = ColumnDataSource(df_centro_week)
p.vbar(x="day_of_week", top="mean_intensity_week", source=source, width=0.5, legend_label="Madrid Central", color=DISTRICT_COLORS[0])

source = ColumnDataSource(df_week)
p.line(x="day_of_week", y="mean_intensity_week", source=source, line_width=3, legend_label="Madrid", color=DISTRICT_COLORS[1])

# Hover tooltip
TOOLTIPS = [
    ("Average Intensity per day", "@mean_intensity_week"),
    ("Day of the Week", "@day_of_week")
    ]
p.add_tools(HoverTool(tooltips=TOOLTIPS, mode="vline"))

p.add_layout(p.legend[0], "right")

if SAVE_PLOTS:
    output_file("html_plots/weekly_analysis_in_out.html", title="Average traffic intensity per day in a week in and out of Madrid Central")
    save(p)

    reset_output()

    output_notebook()
else:
    show(p)

Loading BokehJS ...

Lastly, what we can get from the weekly plot is that **Madrid Central** has a higher average of traffic intensity per day during the weekends than the rest of the city. Also that throughout the week the average increases progressively in both cases.

## 3. Data analysis

Once we have done our exploratory analysis, it is important to obtain conclusions doing a more in depth analysis of the relevant information obtained.

**Air Quality Data Analysis**

**Traffic Data Analysis**

In our exploratory analysis we visualized a static map. But to see if there is a difference we have to analyze if there is a significant difference between before the regulations and after.

In [86]:
traffic_points

,idelem,tipo_elem,cod_cent,name,st_x,st_y,latitude,longitude,utm_x,utm_y,district
0,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06,Moncloa - Aravaca
1,3600,494,PM30901,PM30901,443729.047369,4.473268e+06,40.408129,-3.663184,-407783.811885,4.925429e+06,Moratalaz
2,3705,494,PM41451,PM41451,439858.261097,4.471574e+06,40.392598,-3.708640,-412843.963659,4.923159e+06,Carabanchel
3,6823,494,PM41453,PM41453,439188.095183,4.470895e+06,40.386431,-3.716471,-413715.710072,4.922257e+06,Carabanchel
4,7033,495,01015,Pº Castellana S-N - Pl. Colon-Hermosilla,441569.555897,4.475502e+06,40.428107,-3.688839,-410639.639249,4.928350e+06,Salamanca
...,...,...,...,...,...,...,...,...,...,...,...
3861,6094,495,47033,Rotonda Nudo Manoteras - Av. Alcalde Conde May...,443873.411055,4.481898e+06,40.485884,-3.662246,-407679.323268,4.936803e+06,Hortaleza
3862,6999,495,55065,Av. Consejo de Europa - Av. Consejo de Europa-...,447857.576632,4.479151e+06,40.461390,-3.615013,-402421.374536,4.933219e+06,Barajas
3863,9939,495,87029,(TACTICO) SALIDA BRICOMART,441776.002171,4.468693e+06,40.366783,-3.685782,-410299.414801,4.919386e+06,Usera
3864,5398,495,94017,DOM. PARRAGA O-E(P? FERROV-TRAV. LEGAN),439308.041175,4.466186e+06,40.344022,-3.714610,-413508.524109,4.916061e+06,Villaverde


In [87]:
df_traffic_points = traffic_points.copy()
df_traffic_points = pd.merge(df_traffic_points,
                             traffic_points_intensity_df.groupby(["idelem", "year", "month"]).agg(intensity=("mean_intensity", "mean")).reset_index(),
                             on="idelem")

df_traffic_points

,idelem,tipo_elem,cod_cent,name,st_x,st_y,latitude,longitude,utm_x,utm_y,district,year,month,intensity
0,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06,Moncloa - Aravaca,2016,1,731.360887
1,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06,Moncloa - Aravaca,2016,2,774.546446
2,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06,Moncloa - Aravaca,2016,3,721.848204
3,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06,Moncloa - Aravaca,2016,4,753.728637
4,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06,Moncloa - Aravaca,2016,5,737.511780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185717,5399,495,94018,DOMIN. PARRAGA E-O(A.PALACIOS-TR. LEGAN),439542.738618,4.466176e+06,40.343945,-3.711846,-413200.815293,4.916050e+06,Villaverde,2019,10,134.905726
185718,5399,495,94018,DOMIN. PARRAGA E-O(A.PALACIOS-TR. LEGAN),439542.738618,4.466176e+06,40.343945,-3.711846,-413200.815293,4.916050e+06,Villaverde,2019,11,133.696440
185719,5399,495,94018,DOMIN. PARRAGA E-O(A.PALACIOS-TR. LEGAN),439542.738618,4.466176e+06,40.343945,-3.711846,-413200.815293,4.916050e+06,Villaverde,2019,12,121.234405
185720,5399,495,94018,DOMIN. PARRAGA E-O(A.PALACIOS-TR. LEGAN),439542.738618,4.466176e+06,40.343945,-3.711846,-413200.815293,4.916050e+06,Villaverde,2020,1,128.942083


In [88]:
map = folium.Map(location=[40.420177, -3.703928], zoom_start=12, tiles='cartodb positron')


heatmap_time_data = defaultdict(list)

for _, row in (df_traffic_points.iterrows()):
    key = str(row["month"]) + " " + str(row["year"])
    heatmap_time_data[key].append([row["latitude"], row["longitude"]])



folium.plugins.HeatMapWithTime(data=list(heatmap_time_data.values()), index=list(heatmap_time_data.keys()) , radius=5, auto_play=True).add_to(map)
map

NameError: name 'folium' is not defined

Ok, this map as informative as we thought it would be. We can see small changes in different districts, but nothing too significative. That is why we will try to carry out a different visualization, in an attempt to display the changes more clearly.

## 4. Genre

### Genre of the data story

We are going to focus on a **Partitioned Poster**, with a special focus on use user interaction. Why? because we believe ...

REVISIT THIS ALL TOGETHER TO DECIDE AND WRITE MORE

### Visual Narrative

**Visual Structuring**: Consistent Visual Platform

**Highlighting**: Close-Ups

**Transition Guidance**: Object Continuity

### Narrative Structure

**Ordering**: User Directed Path

**Interractivity**: Hover Highlighting / Details and Filtering / Selection / Search

**Messaging**: Introductory Text, Captions/Headlines and Summary / Synthesis


## 5. Visualizations

## 6. Discussions

## 7. Contributions

---

# Constants and helper functions

In [ ]:
def utm_from_latlon(lat, lon):
    """ From a given lat and lon, calculates the correct UTM coordinates to 
        plot using `bokeh` 
    """
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale

    return x, y

def get_lat_lon_utm(row):
    """ From a row containing the columns 'st_x' and 'st_y' calculates both the lat and lon
        and the correct UTM coordinates to plot using `bokeh`
    """

    # 30 and 'T' is the zone of Madrid
    lat, lon = utm.to_latlon(row["st_x"], row["st_y"], 30, "T")
    
    x, y = utm_from_latlon(lat, lon)

    return pd.Series([lat, lon, x, y])

In [ ]:
def get_color_from_palette(color):
    """ Getting colors for plotting """
    return tuple([int(c * 255) for c in color])

def get_dark_color_from_palette(color):
    """ Getting darker colors for plotting """
    return tuple([int(c * 200) for c in color])

PALETTE = "colorblind"

In [ ]:
get_color_from_palette(sns.color_palette(PALETTE)[0])

# Madrid Central

We are going to analyze the impact of Madrid Central both from an air quality and a traffic viewpoint.

## Air Quality

In [ ]:
# load air quality stations
df_stations = pd.read_csv('shared_data/air_quality/air_quality_stations.csv')

# load magnitud table
df_magnitud = pd.read_csv('shared_data/air_quality/air_quality_magnitud.csv', sep=';')

# load air quality data
df = pd.read_csv('data/air_quality_data.csv')

# converting Date to datetime type
df["datetime"] = pd.to_datetime(df["datetime"])

# merge with air quality stations
df = pd.merge(df, df_stations, left_on = 'PUNTO_MUESTREO', right_on='punto_muestreo', how='left').drop('PUNTO_MUESTREO', axis=1)

# merge with air quality magnitud
df = pd.merge(df, df_magnitud, left_on = 'MAGNITUD', right_on='magnitud_id', how='left').drop('MAGNITUD', axis=1)

df.head()

In [ ]:
print('AIR QUALITY STATION NAMES:')
print([name for name in df.name.unique()])

In [ ]:
# MAP
# load MC area
cm_points = pd.read_csv('shared_data/districts/central_madrid_points.csv')

points = df_stations[["utm_x", "utm_y"]].values
path = Path(cm_points[["utm_x", "utm_y"]].values)
points_in_path_mask = path.contains_points(points)

df_stations["madrid_central"] = False

df_stations.loc[points_in_path_mask, "madrid_central"] = True

df_stations.head()

In [ ]:
# MAP

# load MC area
cm_points = pd.read_csv('shared_data/districts/central_madrid_points.csv')

points = df_stations[["utm_x", "utm_y"]].values
path = Path(cm_points[["utm_x", "utm_y"]].values)
points_in_path_mask = path.contains_points(points)

# plot map
p = figure(title="Air quality stations in Madrid", x_axis_type="mercator", y_axis_type="mercator")

source = ColumnDataSource(df_stations)
cr = p.circle(x="utm_x", y="utm_y",  size=10, source=source)

cartodb = get_provider(CARTODBPOSITRON)
p.add_tile(cartodb)

p.add_tools(HoverTool(tooltips=[('Name', '@name')], renderers=[cr_in, cr_out]))

# add interactive legend
# legend = Legend(items=[(, [cr_in]), ('OUT of Madrid Central area', [cr_out])], location='center') 
p.legend.click_policy="hide"
# legend.location = "top_left"

p = layout(p, sizing_mode='scale_both')

# output_file("html_plots/air_quality_stations.html", title="Air quality stations")
# save(p)
# reset_output()
# output_notebook()

show(p)

Air quality in different stations

Air quality difference respect previous years

In [ ]:
def get_month_year(aRow):
    return aRow.name.month_name() + ' ' + str(aRow.name.year)

def get_stats_dataframe(station):
    df1 = df[(df.name == station)][['formula','value','datetime']]
    df1 = df1.pivot(index='datetime', columns='formula', values='value').reset_index()
    df1['datetime'] = df1.datetime.dt.floor('D')
    df1['datetime'] = df1['datetime'].apply(lambda dt: dt.replace(day=1))
    tracking_gas = df1.columns.values[1:]
    # cut outliers!!!
    if station == 'Plaza del Carmen':
        df1 = df1[df1.SO2 <500 ]
        df1 = df1[df1.CO < 10 ]
    # homogenization (everything in µg/m^3)
    for aGas in df1.columns.values[1:]:
        unit = df_magnitud[df_magnitud.formula==aGas].unit_per_m3.values[0]
        if ((unit == 'mg') or (unit == '10μg')):
            # we change to 10
            df1[aGas] = df1[aGas] * 100 #*1000 to get it in μg/m3 exactly
            idx = df_magnitud[df_magnitud.formula==aGas].index
            df_magnitud.loc[idx, 'unit_per_m3'] = '10μg'
    # get all stats
    df1_stats = df1.groupby(['datetime']).agg(['mean','std'])
    df1_stats.columns = df1_stats.columns.to_flat_index()
    df1_stats.columns = pd.Index([a+'_'+b for a,b in df1_stats.columns])
    df1_stats['date'] = df1_stats.apply(get_month_year, axis=1)
    for aGas in tracking_gas:
        meanColumn = aGas+'_mean'
        stdColumn = aGas+'_std'
        df1_stats[aGas+'_upper'] = df1_stats[meanColumn]+df1_stats[aGas+'_std']
        df1_stats[aGas+'_lower'] = df1_stats[meanColumn]-df1_stats[aGas+'_std']
    return tracking_gas, df1_stats

def get_bokeh_viz_evolution_over_time(df1_stats, aText, tracking_gas):

    # create annotations for time marks
    startMC = time.mktime(dt(2018, 11, 30, 0, 0, 0).timetuple())*1000
    startMC_span = Span(location=startMC,
                                dimension='height', line_color='black',
                                line_dash='dashed', line_width=2, line_alpha=0.3)

    finesMC = time.mktime(dt(2019, 3, 15, 0, 0, 0).timetuple())*1000
    finesMC_span = Span(location=finesMC,
                                dimension='height', line_color='black',
                                line_dash='dashed', line_width=2, line_alpha=0.3)

    endMC = time.mktime(dt(2019, 7, 1, 0, 0, 0).timetuple())*1000
    endMC_span = Span(location=endMC,
                                dimension='height', line_color='black',
                                line_dash='dashed', line_width=2, line_alpha=0.3)

    cds_stats = ColumnDataSource(data=df1_stats)

    p = figure(
        x_axis_type="datetime",
        width=950,
        height=450,
        title='Evolution of pollutant concentrations over time in '+aText, 
        y_axis_label='Gas Concentration', 
        x_axis_label='Date'
    )

    # create color palette
    colors_gas = dict(zip(tracking_gas,list(bokeh.palettes.brewer['Dark2'][len(tracking_gas)])))

    # add the data of each gas + interactive legend
    lines, circles, bands = {}, {}, {}
    items = [] 
    for aGas in tracking_gas:
        unit = df_magnitud[df_magnitud.formula==aGas].unit_per_m3.values[0]
        # add line of mean
        lines[aGas] = p.line('datetime', aGas+'_mean', source=cds_stats, color = colors_gas[aGas])
        # add dots of mean
        circles[aGas] = p.circle('datetime',aGas+'_mean', source=cds_stats, color=colors_gas[aGas], size=5, alpha=0.5)
        p.add_tools(HoverTool(tooltips=[
            ('Gas',aGas),
            ('Date', '@date'),
            ('Average value', f'@{aGas}_mean {unit}/m3'), 
            ('Standard Deviation', f'@{aGas}_std {unit}/m3')
        ], renderers=[circles[aGas]]))
        # add variance
        bands[aGas] = p.varea(x='datetime', y1=aGas+'_upper', y2=aGas+'_lower', source=cds_stats, fill_alpha=0.1, fill_color=colors_gas[aGas])
        # append legend list
        items.append((f'{aGas} ({unit}/m3)', [lines[aGas], circles[aGas], bands[aGas]]))

    # add legend
    legend = Legend(items=items, location='center') 
    legend.click_policy="hide"
    legend.location = 'top_left'
    p.add_layout(legend)

    # add annotations to plot
    p.add_layout(startMC_span)
    p.add_layout(finesMC_span)
    p.add_layout(endMC_span)
    
    return p

In [ ]:
stations = ['Plaza del Carmen', "Plaza de España", "Castellana", "Retiro", "Méndez Álvaro"]

tabs = []

for station in stations:
    all_tracking_gas, df_stats_station = get_stats_dataframe(station)
    p = get_bokeh_viz_evolution_over_time(df_stats_station, station, all_tracking_gas)

    p = layout(p, sizing_mode='stretch_both')

    tabs.append(Panel(child=p, title=station))

tabs = Tabs(tabs=tabs)

# output_file("html_plots/air_quality_evolution_tabs.html")
# save(tabs)
# reset_output()
# output_notebook()

show(tabs)

## Traffic points

In [ ]:
traffic_points = pd.read_csv("shared_data/traffic_points/pmed_trafico_03052016.csv", sep=";")

traffic_points[["latitude", "longitude", "utm_x", "utm_y"]] = traffic_points.apply(get_lat_lon_utm, axis=1)

traffic_points = traffic_points.rename(columns = {'nombre':'name'})

traffic_points.head()

In [ ]:
with open("shared_data/districts/districts.geojson", "r") as geojson:
    geodata = json.load(geojson)

df_districts = pd.DataFrame([], columns=["name", "latitude",
                                         "longitude", "utm_x",
                                         "utm_y"])
for district in geodata["features"]:
    # Get district name
    district_name = district["properties"]["NOMBRE"]
    
    # Get district coordinates
    district_coord = district["geometry"]["coordinates"][0]
    df_district = pd.DataFrame(district["geometry"]["coordinates"][0], columns=["st_x", "st_y"])
    df_district["name"] = district_name
    
    # Calculate correct utm
    df_district[["latitude", "longitude", "utm_x", "utm_y"]] = df_district.apply(get_lat_lon_utm, axis=1)
    df_district = df_district.drop(columns=["st_x", "st_y"])
    
    # Append to all districts dataframe
    df_districts = pd.concat([df_districts, df_district]).reset_index(drop=True)


district_name = df_districts["name"].unique()
df_districts